In [ ]:
from pyhive import hive
from TCLIService.ttypes import TOperationState


def execute_sql(query):
    
    conn = hive.Connection(host='192.168.10.112',
                           port=10000,
                           auth='CUSTOM',
                           database='default',
                           username='dxg',
                           password='123456',
                           configuration={'hive.exec.reducers.max': '123'}
                      )
    cursor = conn.cursor()
    cursor.execute(query,  async_=True)
    status = cursor.poll().operationState
    while status in (TOperationState.INITIALIZED_STATE, TOperationState.RUNNING_STATE):
        status = cursor.poll().operationState
    res = cursor.fetchall()
    cursor.close()
    conn.close()
    return res

In [ ]:
types = []
        
res1 = execute_sql('select media_tags from cartoon_info')

for i in res1:
    for j in i[0].split(','):
        if j!='' and j not in types:
            types.append(j)

# for type_ in types:
#     sql=f'SELECT SUM(views) `播放量`,cast(sum(views)/sum(episodes) as decimal(10,0)) `集均播放量`,sum(coins)  `硬币`, sum(share)  `转发量`, sum(danmakus)  `弹幕量`, sum(likes)  `点赞数` ,sum(reply) `评论数`FROM cartoon_info where locate("{type_}",media_tags)>0'

#     print("*"*10 +type_+"风格 查询结果"+ "* "*10)
#     res = execute_sql(sql)
#     print(res)


# 玫瑰图  pyecharts 

In [ ]:

import pandas as pd
from pyecharts import options as opts
from pyecharts.charts import Pie

from pyecharts.globals import CurrentConfig
from pyecharts.charts import Radar
from pyecharts.globals import ThemeType, SymbolType
sql_num = "SELECT cartoon, follow FROM cartoon_info where score> 9 limit 10 "
v3=[]
v4=[]
sh = execute_sql(sql_num)
for s in sh:
    v3.append(s[0])
    v4.append(s[1])

data = {'番剧':v3,
       '追剧人数':v4}
df = pd.DataFrame(data)
color_series = ['#FAE927','#E9E416','#C9DA36','#9ECB3C','#6DBC49',
                '#37B44E','#3DBA78','#14ADCF','#209AC9','#1E91CA',
                '#2C6BA0','#2B55A1','#2D3D8E','#44388E','#6A368B'
                '#7D3990','#A63F98','#C31C88','#D52178','#D5225B',
                '#D02C2A','#D44C2D','#F57A34','#FA8F2F','#D99D21',
                '#CF7B25','#CF7B25','#CF7B25']

v = df['番剧'].values.tolist()
d = df['追剧人数'].values.tolist()
pie1 = Pie(init_opts=opts.InitOpts(width='1250px', height='750px'))
pie1.set_colors(color_series)

pie1.add("", [ list(z) for z in zip(v, d) ],
        radius=["30%", "100%"],
        center=["50%", "50%"],
        rosetype="area"
        )
pie1.set_global_opts(title_opts=opts.TitleOpts(title='B站番剧名称',subtitle='追剧人数',
                                               title_textstyle_opts=opts.TextStyleOpts(font_size=25,color= '#0085c3'),
                                               subtitle_textstyle_opts= opts.TextStyleOpts(font_size=50,color= '#003399'),
                                               pos_right= 'center',pos_left= 'center',pos_top='42%',pos_bottom='center'
                                              ),
                     legend_opts=opts.LegendOpts(is_show=False),
                     toolbox_opts=opts.ToolboxOpts())

pie1.set_series_opts(label_opts=opts.LabelOpts(is_show=True, position="inside", font_size=12,
                                               formatter="{b}有:{c}人", font_style="italic",
                                               font_weight="bold", font_family="SimHei"
                                               ),
                     )
pie1.render("html/玫瑰图.html")

# 词云图  pyecharts 

In [ ]:
import pandas as pd
import jieba
from collections import Counter
import pyecharts.options as opts
from pyecharts.charts import WordCloud
from pyecharts.charts import Radar
sql_num1 = "SELECT message FROM cartoon_comments  "

v1=[]
sh = execute_sql(sql_num1)
for s in sh:
    v1.append(s[0])

text = str(v1)
def split_word(text):
    word_list = list(jieba.cut(text))
    # 去掉一些无意义的词和符号，我这里自己整理了停用词库
    with open('res/停用词库.txt',encoding='utf-8') as f:
        meaningless_word = f.read()
    result = []
    # 筛选词语
    for i in word_list:
        if i not in meaningless_word:
            result.append(i.replace(' ', ''))
    return result
def word_counter(words):
    # 词频统计,使用Count计数方法
    words_counter = Counter(words)
    # 将Counter类型转换为列表
    words_list = words_counter.most_common(2000)
    return words_list
def word_cloud(data):
    (
        WordCloud()
            .add(series_name="",
                 # 添加数据
                 data_pair=data,
                 # 字间隙
                 word_gap = 5,
                 # 调整字大小范围
                 word_size_range=[15, 80],
                 shape="cursive",
                 mask_image='res/haha.jpg'
                 )
            .set_global_opts(
            title_opts=opts.TitleOpts(
                title="", title_textstyle_opts=opts.TextStyleOpts(font_size=23)
            ),
            tooltip_opts=opts.TooltipOpts(is_show=True),
        )
        # 输出为html格式
            .render("html/词云图.html")
    )
def main():
    
    words = split_word(text)
    data = word_counter(words)
    word_cloud(data)

if __name__ == '__main__':

    main()


# 雷达图  pyecharts 

In [ ]:
from decimal import Decimal
 
types = ['漫画改', '搞笑', '战斗', '日常', '恋爱', '校园', '小说改', '奇幻', '热血', '原创', '魔法', '冒险', '架空', '特摄', '推理', '智斗', '声控', '治愈', '萌系', '游戏改', '励志', '职场', '催泪', '科幻', '机战', '少女', '运动']
results1 = [
    (5455666831, Decimal('2001345'), 26460760, 3730794, 360231774, 45148993, 4540252)
,
(736697275, Decimal('13394496'), 4935167, 818652, 10537308, 7053158, 1017123)
,
(4993057128, Decimal('9583603'), 22205502, 2796083, 45266504, 35377700, 4947353)
,
(765648165, Decimal('11259532'), 4208682, 827263, 11365449, 5888429, 862032)
,
(741391936, Decimal('13479853'), 8135526, 1110559, 11416516, 8754884, 1073817)
,
(750261957, Decimal('13397535'), 8171909, 1108659, 11493890, 8867238, 1084694)
,
(2302614405, Decimal('16565571'), 9721457, 936759, 24014122, 12563093, 2754450)
,
(3159885659, Decimal('10821526'), 14152867, 1762863, 28366623, 22579139, 3319491)
,
(3175697499, Decimal('5189048'), 14010324, 1877153, 29443915, 23125946, 3165148)
,
(9556634, Decimal('530924'), 38258, 28627, 44537, 160519, 13111)
,
(1814731829, Decimal('6435219'), 8607561, 663271, 18882322, 11804731, 2191402)
,
(760098174, Decimal('2714636'), 3159442, 543667, 7126540, 6504751, 891774)
,
(1089603456, Decimal('3572470'), 3287454, 503218, 13221019, 3761786, 1440158)
,
(12465605, Decimal('1246561'), 71351, 23827, 113259, 233413, 17406)
,
(1064628492, Decimal('484804'), 1434049, 237908, 319414656, 3956475, 531207)
,
(2168184932, Decimal('959374'), 6075758, 806767, 332511415, 7843396, 2254591)
,
(850264402, Decimal('32702477'), 3876823, 498836, 9988301, 3798199, 526374)
,
(176984522, Decimal('5056701'), 2132569, 388853, 3534973, 1954898, 516693)
,
(433289695, Decimal('12379706'), 1768880, 300134, 6195375, 3210578, 357378)
,
(7399124, Decimal('822125'), 42696, 5096, 41850, 191032, 17358)
,
(628332426, Decimal('11424226'), 4560274, 787741, 5827627, 6909680, 706290)
,
(157237232, Decimal('11231231'), 1964153, 351980, 3363158, 1430386, 486611)
,
(157237232, Decimal('11231231'), 1964153, 351980, 3363158, 1430386, 486611)
,
(1319248, Decimal('188464'), 4265, 1734, 11283, 24638, 3044)
,
(1319248, Decimal('188464'), 4265, 1734, 11283, 24638, 3044)
,
(213269315, Decimal('16405332'), 3530911, 369188, 2374159, 4780685, 191303)
,
(3372137, Decimal('337214'), 18282, 8275, 25881, 78275, 4345)
]

In [ ]:

import pandas
from pyecharts import options as opts
from pyecharts.charts import Pie
import decimal
from pyecharts.globals import CurrentConfig
from pyecharts.charts import Radar
from pyecharts.globals import ThemeType, SymbolType
a1=[]        
types = []
res1 = execute_sql('select media_tags from cartoon_info')

for i in res1:
    for j in i[0].split(','):
        if j!='' and j not in types:
            types.append(j)
radar = Radar()
radar.add_schema(
    schema = [
        opts.RadarIndicatorItem(name = '播放量', max_ = 7123456000),
        opts.RadarIndicatorItem(name = '集均播放量', max_ = 51234560),
        opts.RadarIndicatorItem(name = '硬币', max_ = 51234560),
        opts.RadarIndicatorItem(name = '转发量', max_ = 5123456),
        opts.RadarIndicatorItem(name = '弹幕量', max_ = 512345600),
        opts.RadarIndicatorItem(name = '点赞数', max_ = 51234560),
        opts.RadarIndicatorItem(name = '评论数', max_ = 5123456),
    ]
)   

for j in range(len(types)):
    data_ = []
    a1 = results1[j]
    for k in a1:
        if type(k)==decimal.Decimal:
            data_.append(float(k))
            continue
        data_.append(k)
        
        numbers=[]
        numbers = [ int(x) for x in data_ ]
        list01=[]
        list01.append(numbers)
        radar.add(types[j],list01)

radar.set_series_opts(label_opts = opts.LabelOpts(is_show = False))

radar.render('html/雷达图.html')

# 条形图  matplotlib

In [ ]:

import matplotlib.pyplot as plt
import matplotlib
import numpy as np


matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['axes.unicode_minus'] = False


sql1="SELECT cartoon FROM cartoon_info where score> 9 limit 5 "
list=[]
sh = execute_sql(sql1)
for s in sh:
    list.append(s[0])
sql2="SELECT likes FROM cartoon_info where score> 9 limit 5 "

sh2 = execute_sql(sql2)

list2=[]
for s in sh2:
    list2.append(s[0])
sql3="SELECT share FROM cartoon_info where score> 9 limit 5 "

sh3 = execute_sql(sql3)

list3=[]
for s in sh3:
    list3.append(s[0])

bar_width = 0.5
index_male = np.arange(len(list)) 
index_female = index_male + bar_width 

plt.bar(index_male, height=list2, width=bar_width, color='b', label='点赞量')
plt.bar(index_female, height=list3, width=bar_width, color='g', label='转发量')

plt.legend()  
plt.xticks(index_male + bar_width/2, list,fontsize=10)  
plt.xticks(rotation=-90,ha='right')
plt.title('评分前五动漫的点赞量和转发量') 
plt.savefig("html/条形图.png")
plt.show()


# 树形图 matplotlib

In [ ]:
import pandas as pd
import  matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
from  matplotlib.font_manager import FontProperties
import squarify
mpl.rcParams['font.sans-serif'] = ['SimHei']

list4=[]
for i in results1:
    list4.append(i[0])

list5=[]
list5=list4[0:9]

df1=pd.Series(list5)
list6=[]
list6=types[0:9]
df2=pd.Series(list6)
plt.rcParams["font.size"]=16
plt.figure(figsize=(15,9))
colors=['steelblue','red','indianred','green','yellow','orange','pink','red','green','yellow']
plot=squarify.plot(
    sizes=df1,
    label=df2,
    color=colors,
    alpha=0.6,
    value=df1,
    edgecolor='white',
    linewidth=8)
plt.rc('font',size=15)
plot.set_title('每一类动漫播放量',fontdict={'fontsize':20})
plt.axis('off')
plt.tick_params(top='off',right='off')
plt.savefig("html/树形图.png")
plt.show()

# 火柴杆图 matplotlib

In [ ]:
import pandas as pd
import  matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
from  matplotlib.font_manager import FontProperties
import squarify
mpl.rcParams['font.sans-serif'] = ['SimHei']
sql1="select  cartoon,score*10 from cartoon_info where score >9   limit 5  "
list7=[]
list8=[]

sh = execute_sql(sql1)

for s in sh:
    list7.append(s[0])
    list8.append(int(s[1]))
plt.rcParams["font.size"]=10
x =list7
y = list8
bottom = -0.1
hold = False
plt.figure(figsize=(10,7))
label = '动漫评分百分制'
markerline, stemlines, baseline = plt.stem(x, y, bottom=bottom,label=label)
plt.xticks(rotation=90,ha='right')
plt.setp(markerline, color='red', marker='o')
plt.setp(stemlines, color='blue', linestyle=':')
plt.setp(baseline, color='grey', linewidth=2, linestyle='-')
plt.xlabel('动漫名称')
plt.ylabel('分数')
plt.title('动漫评分前五')
plt.ylabel('')
plt.legend()
plt.savefig("html/火柴杆图.png")
plt.show()

# 子图 matplotlib

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import squarify
from matplotlib.pyplot import MultipleLocator
matplotlib.rcParams['font.sans-serif']=['SimHei']   
matplotlib.rcParams['axes.unicode_minus']=False
sql_1="select cartoon,coins,reply,episodes,count,follow from cartoon_info where score > 9 limit 6"

sh = execute_sql(sql_1)
list11=[]
list22=[]
list33=[]
list44=[]
list55=[]
list66=[]
for i in sh:
    list11.append(i[0])
    list22.append(i[1])
    list33.append(i[2])

    list44.append(i[3])
    list55.append(i[4])
    list66.append(i[5])

plt.figure(figsize=(20,8))     

plt.subplot(231)
plt.xticks(rotation=90,ha='right')
data1 = list11
data2 = list22
data3 = list33
data4 = list44
data5 = list55
data6 = list66

bar_width = 0.3


plt.bar(data1, data3, label = '动漫评论数', color = 'steelblue', alpha = 0.8)
plt.bar(data1, data5, label = '动漫总计评分人数', color = 'indianred', alpha = 0.8)


plt.title('动漫评论数与总计评分人数')

plt.subplot(232)
plt.xticks(rotation=90,ha='right')

plt.plot(data1, data4,c='red', alpha=0.5)

plt.xlabel('动漫名称')
plt.ylabel('动漫集数')
plt.title('动漫集数')


plt.subplot(233)

plot=squarify.plot(
    sizes=data6,
    label=data1,
    alpha=0.6,
    value=data6,
    edgecolor='white',
    linewidth=8
       )

plt.title('动漫播放量分析')

plt.suptitle('动漫分析', fontsize=12)

plt.subplots_adjust(left=None, bottom=None, right=None, top=None,wspace=0.5, hspace=0.5)
plt.savefig('html/子图.png')
plt.show()


# 散点图 pyecharts 

In [ ]:
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK
from pyecharts.globals import ThemeType, SymbolType
from pyecharts import options as opts
from pyecharts.charts import EffectScatter, Page
ha="select score,cast(avg(`like`) as int)  from cartoon_info a  join cartoon_comments b  where a.ssid=b.ssid and score!=-1.0  group by score order by score DESC"
v1 = []
v2 = []

sh = execute_sql(ha)
for s in sh:
    v1.append(s[0])
    v2.append(s[1])

list01= [ str(i)  for i in v1[0:4] ]

list02=v2[0:4]


def effectscatter_splitline() -> EffectScatter:
    c = (
        EffectScatter()
        .add_xaxis(list01)
        .add_yaxis("",list02, symbol=SymbolType.ARROW)
        .set_global_opts(
            title_opts=opts.TitleOpts(title="不同评分的点赞评论数", subtitle="动漫分析"),
            xaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=True)),
            yaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=True)),
            toolbox_opts=opts.ToolboxOpts(),
            legend_opts=opts.LegendOpts(is_show=True)

        )
    )

    return c


effectscatter_splitline().load_javascript()

effectscatter_splitline().render("html/散点图.html")


# 漏斗图 pyecharts 

In [ ]:
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

from pyecharts import options as opts
from pyecharts.charts import Funnel, Page
ha1="select cartoon,follow  from cartoon_info  where score >9  limit 5"
v111 = []
v222 = []

sh1 = execute_sql(ha1)
for s in sh1:
    v111.append(s[0])
    v222.append(s[1])
#画漏斗图
def funnel_label() -> Funnel:
    c = (
        Funnel()
        .add("追番人数",
            [list(z) for z in zip(v111, v222)],
            sort_="descending",               #默认是sort_="descending"即从大到小，也可以设置为ascending即反向漏斗
            label_opts=opts.LabelOpts(position="right",formatter='{b} 追番人数 : {c}'),
            gap=2,
            tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{a} <br/>{b} 追番人数 : {c}"),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="动漫追番人数", subtitle="分析"),
            xaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=True)),
            yaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=True)),
            toolbox_opts=opts.ToolboxOpts(),
            legend_opts=opts.LegendOpts(is_show=True)

        )
    )
    return c

#第一次渲染时候调用load_javasrcript文件
funnel_label().load_javascript()
#展示数据可视化图表
funnel_label().render('html/漏斗图.html')


# 环图 pyecharts 

In [ ]:
import pandas as pd
from pyecharts.charts import Pie
from pyecharts import options as opts
 
haha="select cartoon,`reply`  from cartoon_info where score > 9 limit 5"

v01 = []
v02 = []

sh = execute_sql(haha)

for s in sh:
    v01.append(s[0])
    v02.append(s[1])

pie = (
       Pie()
       .add('', [list(z) for z in zip(v01, v02)],radius=[80,150])
       .set_global_opts(title_opts=opts.TitleOpts(title="动漫分析",
                                                  subtitle="评论数分析",
                                                  pos_left="center",
                                                  pos_bottom="center"),
                        legend_opts=opts.LegendOpts(pos_left="left",
                                                    orient="vertical"),
       )
       )

pie.render("html/环图.html")    

## 将分析图组成HTML页面

In [ ]:
head = '''
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>cartoon visualization</title>
    <script type="text/javascript" src="https://assets.pyecharts.org/assets/echarts.min.js"></script>
    <script type="text/javascript" src="https://assets.pyecharts.org/assets/echarts-wordcloud.min.js"></script>
    <script>function refresh(){window.location.reload();}function refresh2(){if(sessionStorage.getItem("isReload")){ console.log("页面被刷新");}else{ console.log("首次被加载");sessionStorage.setItem("isReload", true);setTimeout('refresh()', 100);}}setTimeout('refresh2()', 10);</script>
</head>
<body>
'''
tail = '''
</body>
</html>
'''

In [ ]:
import re

PICS = ["玫瑰图","词云图","雷达图","散点图","漏斗图","环图"]
PICS2 = ['条形图','树形图','火柴杆图','子图']
with open("html/index.html",'w') as f:
    f.write("")
    f.close()
f = open("html/index.html",'a',encoding='utf-8')
f.write(head)
for pic_ in PICS:    
    f1 = open("html/"+pic_+".html",'r',encoding='utf-8')
    res = re.findall('<body>(.*?)</body>',f1.read(),re.S)[0]
    f.write("<h2>"+pic_+"</h2>"+res)
    f1.close()
    print(pic_)
for pic_ in PICS2:
    f.write("<h2>"+pic_+"</h2>"+f'<img src="{pic_}.png">')
    f1.close()
    print(pic_)
f.write(tail)
f.close()